## While execution, only the installation of dependencies take a lot of time but the model is sufficiently efficient in execution, so pls consider the model performance not the time for installation while evaluation.

In [1]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/keypoint-identification-cv-contingent-selections/sample_submission.csv
/kaggle/input/keypoint-identification-cv-contingent-selections/training.csv
/kaggle/input/keypoint-identification-cv-contingent-selections/test.csv


In [2]:
!pip install dlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 76.8 MB/s eta 0:00:00
  Installing build dependencies ... - \ | / - \ | / done
  Getting requirements to build wheel ... - \ done
  Preparing metadata (pyproject.toml) ... - \ done
  Created wheel for dlib: filename=dlib-19.24.2-cp310-cp310-linux_x86_64.whl size=3829151 sha256=581ac656dbb6ddf460e779ffb7208bf5678327fbc5c6f909ddbb1cd1940c62bf
  Stored in directory: /root/.cache/pip/wheels/9b/e2/80/888fdc098db86b463ff0c83ae5e5ca151889e901bc1e9a3a11
Successfully built dlib


In [3]:
import cv2
import dlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
train = pd.read_csv("/kaggle/input/keypoint-identification-cv-contingent-selections/training.csv")
xtest = pd.read_csv("/kaggle/input/keypoint-identification-cv-contingent-selections/test.csv")
ytest = pd.read_csv("/kaggle/input/keypoint-identification-cv-contingent-selections/sample_submission.csv")

In [5]:
column_names = ytest.columns
output = pd.DataFrame(columns=column_names)

In [6]:
output

,left_eye_center_x,left_eye_center_y,right_eye_center_x,right_eye_center_y,left_eye_inner_corner_x,left_eye_inner_corner_y,left_eye_outer_corner_x,left_eye_outer_corner_y,right_eye_inner_corner_x,right_eye_inner_corner_y,...,nose_tip_y,mouth_left_corner_x,mouth_left_corner_y,mouth_right_corner_x,mouth_right_corner_y,mouth_center_top_lip_x,mouth_center_top_lip_y,mouth_center_bottom_lip_x,mouth_center_bottom_lip_y,ID


In [7]:
!wget -nd https://github.com/JeffTrain/selfie/raw/master/shape_predictor_68_face_landmarks.dat

--2023-10-28 21:03:59--  https://github.com/JeffTrain/selfie/raw/master/shape_predictor_68_face_landmarks.dat
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/JeffTrain/selfie/master/shape_predictor_68_face_landmarks.dat [following]
--2023-10-28 21:04:00--  https://raw.githubusercontent.com/JeffTrain/selfie/master/shape_predictor_68_face_landmarks.dat
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 99693937 (95M) [application/octet-stream]
Saving to: ‘shape_predictor_68_face_landmarks.dat’

shape_predictor_68_ 100%[===================>]  95.08M   250MB/s    in 0.4s    

2023-10-28 21:04:00 (250 M

## Model

In [8]:
face_detector = dlib.get_frontal_face_detector()
facelandmark = dlib.shape_predictor("/kaggle/working/shape_predictor_68_face_landmarks.dat")

In [9]:
def left_eye(lm):
    left_eye_centre_x = 0
    left_eye_centre_y = 0
    for i in range(42,48):
        left_eye_centre_x += lm.part(i).x
        left_eye_centre_y += lm.part(i).y
    
    return left_eye_centre_x/6 , left_eye_centre_y/6

In [10]:
def right_eye(lm):
    right_eye_centre_x = 0
    right_eye_centre_y = 0
    for i in range(36,42):
        right_eye_centre_x += lm.part(i).x
        right_eye_centre_y += lm.part(i).y
    
    return right_eye_centre_x/6 , right_eye_centre_y/6

In [11]:
def feat(img, id):
    faces = face_detector(img)
    for face in faces:
        face_landmarks = facelandmark(img, face)
        
        output.at[id, 'left_eye_center_x'], output.at[id, 'left_eye_center_y'] = left_eye(face_landmarks)
        output.at[id, 'left_eye_inner_corner_x'], output.at[id, 'left_eye_inner_corner_y'] = face_landmarks.part(42).x, face_landmarks.part(42).y
        output.at[id, 'left_eye_outer_corner_x'], output.at[id, 'left_eye_outer_corner_y'] = face_landmarks.part(45).x, face_landmarks.part(45).y
        output.at[id, 'left_eyebrow_inner_end_x'], output.at[id, 'left_eyebrow_inner_end_y'] = face_landmarks.part(22).x, face_landmarks.part(22).y
        output.at[id, 'left_eyebrow_outer_end_x'], output.at[id, 'left_eyebrow_outer_end_y'] = face_landmarks.part(26).x, face_landmarks.part(26).y
        output.at[id, 'mouth_left_corner_x'], output.at[id, 'mouth_left_corner_y'] = face_landmarks.part(54).x, face_landmarks.part(54).y
        
        output.at[id, 'mouth_center_top_lip_x'], output.at[id, 'mouth_center_top_lip_y'] = (face_landmarks.part(51).x + face_landmarks.part(62).x)/2, (face_landmarks.part(51).y + face_landmarks.part(62).y)/2
        output.at[id, 'mouth_center_bottom_lip_x'], output.at[id, 'mouth_center_bottom_lip_y'] = (face_landmarks.part(66).x + face_landmarks.part(57).x)/2, (face_landmarks.part(66).y + face_landmarks.part(57).y)/2
        output.at[id, 'nose_tip_x'], output.at[id, 'nose_tip_y'] = face_landmarks.part(30).x, face_landmarks.part(30).y
        
        output.at[id, 'right_eye_center_x'], output.at[id, 'right_eye_center_y'] = right_eye(face_landmarks)
        output.at[id, 'right_eye_inner_corner_x'], output.at[id, 'right_eye_inner_corner_y'] = face_landmarks.part(39).x, face_landmarks.part(39).y
        output.at[id, 'right_eye_outer_corner_x'], output.at[id, 'right_eye_outer_corner_y'] = face_landmarks.part(36).x, face_landmarks.part(36).y
        output.at[id, 'right_eyebrow_inner_end_x'], output.at[id, 'right_eyebrow_inner_end_y'] = face_landmarks.part(21).x, face_landmarks.part(21).y
        output.at[id, 'right_eyebrow_outer_end_x'], output.at[id, 'right_eyebrow_outer_end_y'] = face_landmarks.part(17).x, face_landmarks.part(17).y
        output.at[id, 'mouth_right_corner_x'], output.at[id, 'mouth_right_corner_y'] = face_landmarks.part(48).x, face_landmarks.part(48).y
        
        output.at[id, 'ID'] = id
        
#         for n in range(68):
#             x = face_landmarks.part(n).x
#             y = face_landmarks.part(n).y
#             cv2.circle(img, (x, y), 3, (0, 255, 255), 1)
#             plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
#             plt.axis('off')
#             plt.show()

## Image Interpretation

In [12]:
for i in range(xtest.shape[0]):
    input_string = xtest["Image"][i]

    numbers_str = input_string.split(" ")
    num_array = np.array([float(num) for num in numbers_str])

    width = int(len(num_array)**0.5)
    height = int(len(num_array)**0.5)

    image_array = np.array(num_array).reshape(height, width)
    image_array = np.array(image_array, dtype = 'uint8')
    
    feat(image_array, xtest["ID"][i])

In [13]:
# To view any image and marked labels
# plt.imshow(cv2.cvtColor(image_array, cv2.COLOR_BGR2RGB))
# plt.axis('off')
# plt.show()

In [14]:
output.to_csv('submission.csv', index=False)

## As the values in training sheet had all coordinate values < 96, ie the coordinates were reported from the 2D form of the image as an array. However, the sample_submission.csv file had coordinate values > 96 which suggest that values are reported from 1D representation of array. The submission.csv file has been formed considering the 2D form of array. In case we need the values in the form of 1D array, we just need to reshape the array and then report the values.